# Implementing a piston in a sphere in FriCAS
- Thejasvi Beleyur

This notebook shows my attempts at implementing the model in Chp. 12 of Mellow & Beranek. The model relies on getting the coefficients from the $A_{n}$ 1D matrix, which comes from solving: 

$M.a=b$ --> $a=M^{-1}.b$ (eqn. 12.102)


The matrix $M$ is given by (where $m=0,1...N$ and $n=0,1...N$ ):

$M(m+1,n+1) = \frac{I_{mn}+ \left( nh_{n-1}^{(2)}(kR)-(n+1)h_{n+11}^{(2)}(kR) \right)K_{mn}}{2n+1}$ (eqn. 12.103)

$b(m+1) = -jL_{m}$ (eqn. 12.104)

$a(n+1) = A_{n}$ (eqn. 12.105)

So far, I don't expect $K_{mn}$ and $L_{mn}$ to be a problem as they both produce real-valued output. 

## Implementing $I_{mn}$
The $I_{mn}$ term is a rather involved integral that needs to be calculated numerically as it doesn't have an analytical solution. 

$I_{mn} = \int_0^\alpha \left\{\left( nh_{n-1}^{(2)}(kr_{1}) - (n+1)h_{n+1}^{(2)}(kr_{1} \right)P_{n}(\cos\theta)\cos\theta + n(n+1)h_{n}^{(2)}(kr_{1})(P_{n-1}(\cos\theta)-P_{n+1}(\cos\theta))/kr_{1} \right\}P_{m}(\cos\theta)\frac{r_{1}^2}{R^{2}}tan\theta\:d\theta$

Here, $h_{n}^{(2)}(z)$ is the spherical Hankel function of the second kind: 

$h_{n}^{(2)}(z) = j_{n}(z)-iy_{n}(z)$, where $j_{n},y_{n}$ are the spherical Bessel's functions of the first and second kind. 


### The problem: $I_{mn}$ term produces a complex result
The [numerical quadrature](https://fricas.github.io/api/NumericalQuadrature.html?highlight=aromberg) help page documents a series of methods available to numerically integrate functions which produce ```Float``` outputs. When I try integrating the $I_{mn}$ term numerically it fails. Below is my attempt.

In [1]:
)set output tex on
)set output algebra off

In [2]:
++  Some of the constant values required for the rest of the functions below 
++ These correspond to the acoustics of the problem. 

freq := 50000;
k := 2*%pi/(330/freq);
ka := 5 ;
a := ka/k;
alpha := %pi/3;
R:= a/sin(alpha);
R := numeric(R)::DoubleFloat;
r1 := R*cos(alpha)/cos(theta);
NN :=  3 ;--12 + 2*ka/sin(alpha)--


PositiveInteger

Pi

PositiveInteger

Pi

Pi

Expression ( Integer )

DoubleFloat

Expression ( DoubleFloat )

PositiveInteger

In [3]:
++ I couldn't find inbuilt spherical bessel's functions, and so had to make custom functions 

-- spherical bessel function-- 
sphBessel: (PositiveInteger, DoubleFloat) -> Complex DoubleFloat
sphBessel(n,z) == besselJ(n+0.5,z)*sqrt(%pi/(2*z));
-- spherical neumann function --
sphNeumann: (PositiveInteger, DoubleFloat) -> Complex DoubleFloat
sphNeumann(n,z) == besselY(n+0.5,z)*sqrt(%pi/(2*z));

sphHankel2: (Integer, DoubleFloat) -> Complex(DoubleFloat)
sphHankel2(n,z) == sphBessel(n,z) - %i*sphNeumann(n,z);

Void

Void

Void

Void

Void

Void

In [4]:

alternate_hankels: (PositiveInteger, DoubleFloat) -> Complex(DoubleFloat)
alternate_hankels(n,theta) ==   
    r1 := R*cos(alpha)/cos(theta)
    legendreP(n,cos(theta))*cos(theta) + (n+1)*sphHankel2(n+1,k*r1)

alternate_legendres: (PositiveInteger, DoubleFloat) -> Complex(DoubleFloat)
alternate_legendres(n,theta) ==
    r1 := R*cos(alpha)/cos(theta)
    n*(n+1)*sphHankel2(n,k*r1)*(legendreP(n-1,cos(theta)-legendreP(n+1,cos(theta))))/(k*r1)

legendretan: (PositiveInteger, DoubleFloat)-> Complex(DoubleFloat)
legendretan(m,theta) ==
    r1 := R*cos(alpha)/cos(theta)
    legendreP(m,cos(theta))*(r1^2/R^2)*tan(theta)

one_imn_term: (NonNegativeInteger, NonNegativeInteger, DoubleFloat) -> Complex(DoubleFloat)
one_imn_term(m,n,theta) == 
    r1 := R*cos(alpha)/cos(theta)
    (alternate_hankels(n,theta)+alternate_legendres(m,theta))*legendretan(m,theta)

Void

Void

Void

Void

Void

Void

Void

Void

## Integrate $I_{mn}$ for a fixed value of $m,n$
The numerical integration works with only one free variable. 

Let's set m=10, n=20 and integrate over $\theta$

In [5]:
intg_imn_m10n20: (DoubleFloat) -> Complex(DoubleFloat)
intg_imn_m10n20(theta) == one_imn_term(10,20,theta)

Void

Void

In [6]:
++ Check that the output for the Imn term works ++ 
intg_imn_m10n20(0.2)

Compiling function sphBessel with type (PositiveInteger, DoubleFloat) -> 
Complex(DoubleFloat) 
Compiling function sphNeumann with type (PositiveInteger, DoubleFloat) -> 
Complex(DoubleFloat) 
Compiling function sphHankel2 with type (Integer, DoubleFloat) -> Complex(
DoubleFloat) 
Compiling function alternate_hankels with type (PositiveInteger, DoubleFloat)
 -> Complex(DoubleFloat) 
Your expression cannot be fully compiled because it contains an integer 
expression (for - ) whose sign cannot be determined (in general) and so must 
be specified by you. Perhaps you can try substituting something like
                                  (- :: PI) 
                                     or
                                 (- :: NNI) 
into your expression for - .
FriCAS will attempt to step through and interpret the code.
Compiling function alternate_legendres with type (PositiveInteger, 
DoubleFloat) -> Complex(DoubleFloat) 
Compiling function legendretan with type (PositiveInteger, DoubleFloa

Complex ( DoubleFloat )

In [7]:
aromberg(intg_imn_m10n20,0,alpha,0.99,10^-12,10,50,30)

There are 1 exposed and 0 unexposed library operations named aromberg having 
8 argument(s) but none was determined to be applicable. Use HyperDoc Browse, 
or issue
                            )display op aromberg
to learn more about the available operations. Perhaps package-calling the 
operation or using coercions on the arguments will allow you to apply the 
operation.
Cannot find a definition or applicable library operation named aromberg with 
argument type(s) 
                       FunctionCalled(intg_imn_m10n20)
                             NonNegativeInteger
                                     Pi
                                    Float
                              Fraction(Integer)
                               PositiveInteger
                               PositiveInteger
                               PositiveInteger
 
Perhaps you should use "@" to indicate the required return type, or "$" to 
specify which version of the function you need.
